# <font color='red'>SEDES - Gerador de Dados</font>

# <font color='blue'>1.0 import</font>

In [1]:
import csv
import os
import pandas as pd
import random as rd
import datetime as dt
import names
from dateutil.relativedelta import relativedelta

# <font color='blue'>2.0 parâmetros de ambiente</font>

In [2]:
# campo de negócio no BI
ORGAO_GESTOR = 'orgaoGestorDaPoliticaDeAssistenciaSocial'
ORGANIZACAO_PPA = 'comoEstaOrganizadoOPPADaPASNoMunicipio'
TUP_INSTR_GESTAO = (
    'planoDecenalDeAssistenciaSocial-possui',
    'planoMunicipalDeAssistenciaSocial-possui',
    'PlanoMunicipaldeEducacaoPermanentedoSUAS-possui',
    'diagnosticoSocioterritorial-possui',
    'RelatoriodeGestaoDoExercicioAnterior-possui',
    'LeideRegulamentacaodoSUAS-possui'
)
AREAS_ESSENCIAIS = (
    'quaisAreasEssenciaisDoSUASEstaoImplantadosOficialmente-gestao_do_trabalho',
    'quaisAreasEssenciaisDoSUASEstaoImplantadosOficialmente-protecao_social_basica',
    'quaisAreasEssenciaisDoSUASEstaoImplantadosOficialmente-protecao_social_especial',
    'quaisAreasEssenciaisDoSUASEstaoImplantadosOficialmente-gestao_do_suas'
)
VS = (
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-levanta_dados_no_territorio_equipamento_da_pas',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-analisa_dados_levantados',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-sistematiza_dados_e_produz_relatorios',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-dissemina_dados_e_ou_relatorios',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-realiza_diagnostico_socioterritorial',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-realiza_monitoramento_sistematico_dos_servicos_socioassistenciais',
    'oMunicipioDesenvolveQuaisAcoesDeVigilanciaSocioassistencial-realiza_reunioes_com_as_equipes_responsaveis_pela_execucao_dos_servicos_socioassistenciais'
)
MONITORA_SISTEMAS = (
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-CAD_SUAS',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-RMA',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-SISC',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-E-PCF',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-SIMPETI',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-Plano_de_Acao_SAA',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-Demonstrativo_Sintetico_de_Execucao_Fisico_Financeiro',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-Alimentacao_da_Plataforma_Estadual_de_Monitoramento_da_PAS'
)
ALIMENTA_SUAS = (
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-gestao',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-protecao_social_basica',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-protecao_social_especial',
    'quemAlimentaOsDadosNosSistemasDaRedeSUAS-protecao_social_outros'
)
EQUIPE_VS = (
    {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 'suf_nome': 'nome'},
    {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 'suf_escolaridade': 'escolaridade'},
    {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 'suf_formacao': 'formacao'},
    {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 'suf_cargo': 'cargo'}
)

# data inicial de simulação
DATA_INICIAL = (2018, 3, 1)

# tabelas gerais
N_ORGAO_GESTOR = (6, 11) # quantidade de orgãos gestores possíveis para a tabela de orgãos gestores
N_ESCOLARIDADE = (5, 10) # quantidade de escolaridades possíveis para a tabela escolaridade
N_FORMACAO     = (5, 10) # quantidade de formações possíveis para a tabela formacao
N_CARGO        = (5, 10) # quantidade de cargos possíveis para a tabela cargo
N_EQUIPE_VS    = (2, 6)  # quantidade de pessoas possíveis para compor a equipe de VS

# formulários respondidos
form_resp = (80, 100) # % de formulários respondidos
orgao_gestor = (1, 3) # possibilidades de orgão gestor por município a cada data de referência
campos_por_form = (50, 150) # quantidade de campos por formulário sem contar os identificados para o BI

# <font color='blue'>3.0 extração</font>

## 3.1 extracao de dados externo

In [3]:
# abrindo os arquivos...
caminho_in = '../dados/apoio/'
caminho_out = '../dados/simulacao/'

arq_municipios    = caminho_in + "tab_municipios_new.csv"
arq_instr_gestao  = caminho_in + "tab_instr_gestao.csv"
arq_escolaridades = caminho_in + "tab_escolaridade.csv"
arq_formacoes     = caminho_in + "tab_formacoes.csv"
arq_cargos        = caminho_in + "tab_cargos.csv"

with open(arq_escolaridades) as f:
    lst_tot_escolaridades = list(csv.reader(f))
    
with open(arq_formacoes) as f:
    lst_tot_formacoes = list(csv.reader(f))
    
with open(arq_cargos) as f:
    lst_tot_cargos = list(csv.reader(f))

df_municipios =   pd.read_csv(arq_municipios, sep = ',', dtype = str)
df_instr_gestao = pd.read_csv(arq_instr_gestao, sep = ',', dtype = str)

print("municipios:              linhas = %s , colunas = %s " % df_municipios.shape)
print("instrumentos de gestão:  linhas = %s , colunas = %s " % df_instr_gestao.shape)
print("escolaridades:           linhas = %s " % len(lst_tot_escolaridades))
print("formacoes:               linhas = %s " % len(lst_tot_formacoes))
print("cargos:                  linhas = %s " % len(lst_tot_cargos))

municipios:              linhas = 217 , colunas = 4 
instrumentos de gestão:  linhas = 5 , colunas = 1 
escolaridades:           linhas = 13 
formacoes:               linhas = 226 
cargos:                  linhas = 134 


In [4]:
df_municipios

,cod_ibge,municipio,regiao_planejamento,porte
0,2100055,Açailândia,24 – REGIÃO DOS CARAJÁS,Grande Porte
1,2100105,Afonso Cunha,07 – REGIÃO DO ALTO MUNIM,Porte I
2,2100154,Água Doce Do Maranhão,13 – REGIÃO DO DELTA DO PARNAÍBA,Porte I
3,2100204,Alcântara,20 – REGIÃO DO PERICUMÃ,Porte II
4,2100303,Aldeias Altas,32 – REGIÃO DOS TIMBIRAS,Porte II
...,...,...,...,...
212,2112803,Viana,30 – REGIÃO DOS LAGOS,Porte II
213,2112852,Vila Nova Dos Martírios,24 – REGIÃO DOS CARAJÁS,Porte I
214,2112902,Vitória Do Mearim,26 – REGIÃO DOS EIXOS RODOFERROVIÁRIOS,Porte II
215,2113009,Vitorino Freire,17 – REGIÃO DO MEARIM,Porte II


# <font color='blue'>4.0 transformação</font>

## 4.1 Tabelas de Negócio

### 4.1.1 tabela de "Orgãos Gestores"

In [5]:
# cria uma tabela de orgãos gestores possíveis
# quantidade de orgão gestores na tabela

lst_OG = ['Orgão Gestor ' + str(x + 1).zfill(2) for x in range(rd.randint(N_ORGAO_GESTOR[0], N_ORGAO_GESTOR[1]))]
lst_OG

['Orgão Gestor 01',
 'Orgão Gestor 02',
 'Orgão Gestor 03',
 'Orgão Gestor 04',
 'Orgão Gestor 05',
 'Orgão Gestor 06',
 'Orgão Gestor 07',
 'Orgão Gestor 08']

### 4.1.2 tabela de "Organização de PPAs"

In [6]:
lst_org_PPA = (
    'Por nível de proteção social',
    'Por programas, serviços, projetos e benefícios'
)
lst_org_PPA

('Por nível de proteção social',
 'Por programas, serviços, projetos e benefícios')

### 4.1.3 tabela de "Escolaridade"

In [7]:
# sorteia a quantidade de nomes de escolaridades irão para a lista
n = rd.randint(N_ESCOLARIDADE[0], N_ESCOLARIDADE[1])

# sorteia quais os nomes de escolaridade farão parte dos dados
lst_escolaridade = rd.sample(lst_tot_escolaridades, k=n)
lst_escolaridade

[['Superior - Incompleto'],
 ['Pós-graduação (Lato senso) - Completo'],
 ['Pós-graduação (nível mestrado) - Incompleto'],
 ['Fundamental - Completo'],
 ['Pós-graduação (nível doutor) - Completo'],
 ['Superior - Completo'],
 ['Pós-graduação (nível mestrado) - Completo'],
 ['Médio - Completo'],
 ['Pós-graduação (Lato senso) - Incompleto'],
 ['Médio - Incompleto']]

### 4.1.4 tabela de "Formação"

In [8]:
# sorteia a quantidade de nomes de formações irão para a lista
n = rd.randint(N_FORMACAO[0], N_FORMACAO[1])

# sorteia quais os nomes de escolaridade farão parte dos dados
lst_formacao = rd.sample(lst_tot_formacoes, k=n)
lst_formacao

[['Estudos de Mídia'],
 ['Engenharia de Minas'],
 ['Engenharia Florestal'],
 ['Engenharia Cartográfica e de Agrimensura'],
 ['Agrimensura'],
 ['Engenharia e Produção'],
 ['Jornalismo'],
 ['Fisioterapia'],
 ['Ciências Atuariais']]

### 4.1.5 tabela de "Cargo"

In [9]:
# sorteia a quantidade de nomes de cargos irão para a lista
n = rd.randint(N_CARGO[0], N_CARGO[1])

# sorteia quais os nomes de escolaridade farão parte dos dados
lst_cargo = rd.sample(lst_tot_cargos, k=n)
lst_cargo

[['Mecanico '],
 ['Profissional de engenharia '],
 ['Biomédico Bioquímico '],
 ['Operador '],
 ['Representante comercial '],
 ['Auxiliar de Almoxarifado '],
 ['Auxiliar de Limpeza '],
 ['Designer '],
 ['Pedagogo '],
 ['Padeiro ']]

## (A) Módulo

### A.1 serviços/formulários

In [10]:
# lista de serviços (cada serviço é equivalente e um formulário)
tup_servicos = (
    ('o1-semestral municipio', 'semestral'),
    ('o1-semestral SEDES',     'semestral')
)
df_tb_servicos = pd.DataFrame(tup_servicos, columns=['servico', 'periodicidade'])
df_tb_servicos

,servico,periodicidade
0,o1-semestral municipio,semestral
1,o1-semestral SEDES,semestral


### A.1.1 serviços/formulário x campos fictícios

In [11]:
# lista de campos por serviço/formulário
cont = 0; lst_campos = []
for i in tup_servicos:
    n_campos = rd.randint(10, 15)
    for j in range(n_campos):
        cont += 1
        lst = [i[0], 'col_' + str(cont), 'STRING']
        lst_campos.append(lst)
lst_campos



df_campos = pd.DataFrame(lst_campos, columns=['servico', 'atributo', 'tipo'])
df_campos

,servico,atributo,tipo
0,o1-semestral municipio,col_1,STRING
1,o1-semestral municipio,col_2,STRING
2,o1-semestral municipio,col_3,STRING
3,o1-semestral municipio,col_4,STRING
4,o1-semestral municipio,col_5,STRING
5,o1-semestral municipio,col_6,STRING
6,o1-semestral municipio,col_7,STRING
7,o1-semestral municipio,col_8,STRING
8,o1-semestral municipio,col_9,STRING
9,o1-semestral municipio,col_10,STRING


## (B) Módulo

### B.1 datas base

In [12]:
# cria uma coleção de datas de referências de todos os semestres desde o início até hoje
# confirma que a data inicial é o primeiro dia do semestre
dt_inicial = dt.date(DATA_INICIAL[0], 1 if DATA_INICIAL[1] <= 6 else 7, 1)

# monta uma lista de todos os semestres desde a data inicial até hoje
lst_datas = []; dt_atual = dt_inicial
while dt_atual < dt.date.today():
    dtf = dt_atual + relativedelta(months = 6) - relativedelta(days = 1)
    n_dias = (dtf - dt_atual).days
    lst_datas.append([dt_atual, dtf, n_dias])
    dt_atual = dt_atual + relativedelta(months = 6)

# substitui a última data pela data de hoje
lst_datas[len(lst_datas) - 1][1] = dt.date.today()
lst_datas[len(lst_datas) - 1][2] = (lst_datas[len(lst_datas) - 1][1] - lst_datas[len(lst_datas) - 1][0]).days
lst_datas

[[datetime.date(2018, 1, 1), datetime.date(2018, 6, 30), 180],
 [datetime.date(2018, 7, 1), datetime.date(2018, 12, 31), 183],
 [datetime.date(2019, 1, 1), datetime.date(2019, 6, 30), 180],
 [datetime.date(2019, 7, 1), datetime.date(2019, 12, 31), 183],
 [datetime.date(2020, 1, 1), datetime.date(2020, 6, 30), 181],
 [datetime.date(2020, 7, 1), datetime.date(2020, 12, 31), 183],
 [datetime.date(2021, 1, 1), datetime.date(2021, 6, 30), 180],
 [datetime.date(2021, 7, 1), datetime.date(2021, 7, 6), 5]]

## (C) Módulo

### C.1 municípios

In [13]:
df_municipios = df_municipios[['municipio']]
df_municipios

,municipio
0,Açailândia
1,Afonso Cunha
2,Água Doce Do Maranhão
3,Alcântara
4,Aldeias Altas
...,...
212,Viana
213,Vila Nova Dos Martírios
214,Vitória Do Mearim
215,Vitorino Freire


## (D) Módulo Combinado

### D.1 Formulários Distribuídos - (C.1)-municipios x (A.1)-serviços/formulários

In [14]:
# cria a combinação de municípios e serviços
df = pd.merge(df_tb_servicos, df_municipios, how='cross')
df

,servico,periodicidade,municipio
0,o1-semestral municipio,semestral,Açailândia
1,o1-semestral municipio,semestral,Afonso Cunha
2,o1-semestral municipio,semestral,Água Doce Do Maranhão
3,o1-semestral municipio,semestral,Alcântara
4,o1-semestral municipio,semestral,Aldeias Altas
...,...,...,...
429,o1-semestral SEDES,semestral,Viana
430,o1-semestral SEDES,semestral,Vila Nova Dos Martírios
431,o1-semestral SEDES,semestral,Vitória Do Mearim
432,o1-semestral SEDES,semestral,Vitorino Freire


### D.2 Protocolos/Dt_Protocolos - (D.1) x (B.1)

In [15]:
# cria datas de protocolos
lst = []
pct = 1
for idx in range(df.shape[0]):
    for i in lst_datas:
        n_protocolos = rd.randint(0, 3)
        if n_protocolos > 0:
            lst.append([idx, 'pct_' + str(pct), i[0],
                        df.loc[idx, 'servico'], df.loc[idx, 'municipio']])
            pct += 1
            for j in range(1, n_protocolos):
                dias = rd.randint(1, i[2])
                dt = i[0] + relativedelta(days = dias)
                lst.append([idx, 'pct_' + str(pct), dt, 
                            df.loc[idx, 'servico'], df.loc[idx, 'municipio']])
                pct += 1
lst
df_pct = pd.DataFrame(lst, columns=['idx', 'protocolo', 'dt_protocolo', 'servico', 'municipio'])
df_pct

,idx,protocolo,dt_protocolo,servico,municipio
0,0,pct_1,2018-07-01,o1-semestral municipio,Açailândia
1,0,pct_2,2019-01-01,o1-semestral municipio,Açailândia
2,0,pct_3,2019-02-20,o1-semestral municipio,Açailândia
3,0,pct_4,2019-05-25,o1-semestral municipio,Açailândia
4,0,pct_5,2020-01-01,o1-semestral municipio,Açailândia
...,...,...,...,...,...
5178,433,pct_5179,2018-07-01,o1-semestral SEDES,Zé Doca
5179,433,pct_5180,2019-07-01,o1-semestral SEDES,Zé Doca
5180,433,pct_5181,2019-10-25,o1-semestral SEDES,Zé Doca
5181,433,pct_5182,2021-01-01,o1-semestral SEDES,Zé Doca


### D.3 Campos Fictícios - (D.2) x (A.1.1)

In [16]:
df_final = pd.merge(df_pct, df_campos).drop(['idx', 'municipio'], axis=1)
df_final

,protocolo,dt_protocolo,servico,atributo,tipo
0,pct_1,2018-07-01,o1-semestral municipio,col_1,STRING
1,pct_1,2018-07-01,o1-semestral municipio,col_2,STRING
2,pct_1,2018-07-01,o1-semestral municipio,col_3,STRING
3,pct_1,2018-07-01,o1-semestral municipio,col_4,STRING
4,pct_1,2018-07-01,o1-semestral municipio,col_5,STRING
...,...,...,...,...,...
64767,pct_5183,2021-07-01,o1-semestral SEDES,col_21,STRING
64768,pct_5183,2021-07-01,o1-semestral SEDES,col_22,STRING
64769,pct_5183,2021-07-01,o1-semestral SEDES,col_23,STRING
64770,pct_5183,2021-07-01,o1-semestral SEDES,col_24,STRING


### D.4 Atribuição de Valores aos Campos Fictícios - (D.2) x (A.1.1)

In [17]:
def GeraValor(tp):
    vl = '0'
    if tp == 'STRING':
        vl = 'valor_' + str(rd.randint(1, 1000))
    return vl

df_final['valor'] = df_final['tipo'].apply(GeraValor)
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_1,2018-07-01,o1-semestral municipio,col_1,STRING,valor_288
1,pct_1,2018-07-01,o1-semestral municipio,col_2,STRING,valor_260
2,pct_1,2018-07-01,o1-semestral municipio,col_3,STRING,valor_41
3,pct_1,2018-07-01,o1-semestral municipio,col_4,STRING,valor_420
4,pct_1,2018-07-01,o1-semestral municipio,col_5,STRING,valor_330
...,...,...,...,...,...,...
64767,pct_5183,2021-07-01,o1-semestral SEDES,col_21,STRING,valor_262
64768,pct_5183,2021-07-01,o1-semestral SEDES,col_22,STRING,valor_775
64769,pct_5183,2021-07-01,o1-semestral SEDES,col_23,STRING,valor_510
64770,pct_5183,2021-07-01,o1-semestral SEDES,col_24,STRING,valor_777


### D.5 Campos de Município - (A.1) x (C.1)

In [18]:
df_campos_neg = df_pct.copy()
df_campos_neg['atributo'] = 'municipio'
df_campos_neg['tipo'] = 'STRING'
df_campos_neg = df_campos_neg.rename(columns={'municipio': 'valor'})
df_campos_neg = df_campos_neg.drop(['idx'], axis=1)
df_campos_neg

,protocolo,dt_protocolo,servico,valor,atributo,tipo
0,pct_1,2018-07-01,o1-semestral municipio,Açailândia,municipio,STRING
1,pct_2,2019-01-01,o1-semestral municipio,Açailândia,municipio,STRING
2,pct_3,2019-02-20,o1-semestral municipio,Açailândia,municipio,STRING
3,pct_4,2019-05-25,o1-semestral municipio,Açailândia,municipio,STRING
4,pct_5,2020-01-01,o1-semestral municipio,Açailândia,municipio,STRING
...,...,...,...,...,...,...
5178,pct_5179,2018-07-01,o1-semestral SEDES,Zé Doca,municipio,STRING
5179,pct_5180,2019-07-01,o1-semestral SEDES,Zé Doca,municipio,STRING
5180,pct_5181,2019-10-25,o1-semestral SEDES,Zé Doca,municipio,STRING
5181,pct_5182,2021-01-01,o1-semestral SEDES,Zé Doca,municipio,STRING


### D.6 Concatena Dataset de Campos Fictícios com Campos de Município - (D.4) + (D.5)

In [19]:
df_final = pd.concat([df_final, df_campos_neg])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_1,2018-07-01,o1-semestral municipio,col_1,STRING,valor_288
1,pct_1,2018-07-01,o1-semestral municipio,col_2,STRING,valor_260
2,pct_1,2018-07-01,o1-semestral municipio,col_3,STRING,valor_41
3,pct_1,2018-07-01,o1-semestral municipio,col_4,STRING,valor_420
4,pct_1,2018-07-01,o1-semestral municipio,col_5,STRING,valor_330
...,...,...,...,...,...,...
5178,pct_5179,2018-07-01,o1-semestral SEDES,municipio,STRING,Zé Doca
5179,pct_5180,2019-07-01,o1-semestral SEDES,municipio,STRING,Zé Doca
5180,pct_5181,2019-10-25,o1-semestral SEDES,municipio,STRING,Zé Doca
5181,pct_5182,2021-01-01,o1-semestral SEDES,municipio,STRING,Zé Doca


## (E) Módulo Linhas de Negócio

### E.1 Orgão Gestor

In [20]:
# métrica Órgão Gestor
colunas = ['protocolo', 'dt_protocolo', 'servico']
df_campos_neg = df_pct[(df_pct['servico'] == 'o1-semestral municipio')][colunas]
df_campos_neg['atributo'] = ORGAO_GESTOR
df_campos_neg['tipo'] = 'STRING'
df_campos_neg['valor'] = df_campos_neg.apply(lambda row: rd.sample(lst_OG, k=1)[0], axis=1)

df_final = pd.concat([df_final, df_campos_neg])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_1,2018-07-01,o1-semestral municipio,col_1,STRING,valor_288
1,pct_1,2018-07-01,o1-semestral municipio,col_2,STRING,valor_260
2,pct_1,2018-07-01,o1-semestral municipio,col_3,STRING,valor_41
3,pct_1,2018-07-01,o1-semestral municipio,col_4,STRING,valor_420
4,pct_1,2018-07-01,o1-semestral municipio,col_5,STRING,valor_330
...,...,...,...,...,...,...
2602,pct_2603,2021-01-01,o1-semestral municipio,orgaoGestorDaPoliticaDeAssistenciaSocial,STRING,Orgão Gestor 01
2603,pct_2604,2021-03-04,o1-semestral municipio,orgaoGestorDaPoliticaDeAssistenciaSocial,STRING,Orgão Gestor 04
2604,pct_2605,2021-01-09,o1-semestral municipio,orgaoGestorDaPoliticaDeAssistenciaSocial,STRING,Orgão Gestor 04
2605,pct_2606,2021-07-01,o1-semestral municipio,orgaoGestorDaPoliticaDeAssistenciaSocial,STRING,Orgão Gestor 05


### E.2 Instrumentos de Gestão

In [21]:
# métrica Órgão Gestor
lst_vls = ('false', 'true')
colunas = ['protocolo', 'dt_protocolo', 'servico']

for i in TUP_INSTR_GESTAO:
    df_campos_neg = df_pct[(df_pct['servico'] == 'o1-semestral municipio')][colunas]
    df_campos_neg['atributo'] = i
    df_campos_neg['tipo'] = 'STRING'
    df_campos_neg['valor'] = df_campos_neg.apply(lambda row: rd.sample(lst_vls, k=1)[0], axis=1)
    df_final = pd.concat([df_final, df_campos_neg])

df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_1,2018-07-01,o1-semestral municipio,col_1,STRING,valor_288
1,pct_1,2018-07-01,o1-semestral municipio,col_2,STRING,valor_260
2,pct_1,2018-07-01,o1-semestral municipio,col_3,STRING,valor_41
3,pct_1,2018-07-01,o1-semestral municipio,col_4,STRING,valor_420
4,pct_1,2018-07-01,o1-semestral municipio,col_5,STRING,valor_330
...,...,...,...,...,...,...
2602,pct_2603,2021-01-01,o1-semestral municipio,LeideRegulamentacaodoSUAS-possui,STRING,true
2603,pct_2604,2021-03-04,o1-semestral municipio,LeideRegulamentacaodoSUAS-possui,STRING,false
2604,pct_2605,2021-01-09,o1-semestral municipio,LeideRegulamentacaodoSUAS-possui,STRING,true
2605,pct_2606,2021-07-01,o1-semestral municipio,LeideRegulamentacaodoSUAS-possui,STRING,false


### E.3 Áreas Essenciais

In [22]:
# métrica Áreas Essenciais
lst_vls = ('false', 'true')
colunas = ['protocolo', 'dt_protocolo', 'servico']

for i in AREAS_ESSENCIAIS:
    df_campos_neg = df_pct[(df_pct['servico'] == 'o1-semestral municipio')][colunas]
    df_campos_neg['atributo'] = i
    df_campos_neg['tipo'] = 'STRING'
    df_campos_neg['valor'] = df_campos_neg.apply(lambda row: rd.sample(lst_vls, k=1)[0], axis=1)
    df_final = pd.concat([df_final, df_campos_neg])

df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_1,2018-07-01,o1-semestral municipio,col_1,STRING,valor_288
1,pct_1,2018-07-01,o1-semestral municipio,col_2,STRING,valor_260
2,pct_1,2018-07-01,o1-semestral municipio,col_3,STRING,valor_41
3,pct_1,2018-07-01,o1-semestral municipio,col_4,STRING,valor_420
4,pct_1,2018-07-01,o1-semestral municipio,col_5,STRING,valor_330
...,...,...,...,...,...,...
2602,pct_2603,2021-01-01,o1-semestral municipio,quaisAreasEssenciaisDoSUASEstaoImplantadosOfic...,STRING,true
2603,pct_2604,2021-03-04,o1-semestral municipio,quaisAreasEssenciaisDoSUASEstaoImplantadosOfic...,STRING,false
2604,pct_2605,2021-01-09,o1-semestral municipio,quaisAreasEssenciaisDoSUASEstaoImplantadosOfic...,STRING,false
2605,pct_2606,2021-07-01,o1-semestral municipio,quaisAreasEssenciaisDoSUASEstaoImplantadosOfic...,STRING,false


### E.4 Organização PPA

In [23]:
# métrica Organização de PPA no município
colunas = ['protocolo', 'dt_protocolo', 'servico']
df_campos_neg = df_pct[(df_pct['servico'] == 'o1-semestral municipio')][colunas]
df_campos_neg['atributo'] = ORGANIZACAO_PPA
df_campos_neg['tipo'] = 'STRING'
df_campos_neg['valor'] = df_campos_neg.apply(lambda row: rd.sample(lst_org_PPA, k=1)[0], axis=1)

df_final = pd.concat([df_final, df_campos_neg])
df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_1,2018-07-01,o1-semestral municipio,col_1,STRING,valor_288
1,pct_1,2018-07-01,o1-semestral municipio,col_2,STRING,valor_260
2,pct_1,2018-07-01,o1-semestral municipio,col_3,STRING,valor_41
3,pct_1,2018-07-01,o1-semestral municipio,col_4,STRING,valor_420
4,pct_1,2018-07-01,o1-semestral municipio,col_5,STRING,valor_330
...,...,...,...,...,...,...
2602,pct_2603,2021-01-01,o1-semestral municipio,comoEstaOrganizadoOPPADaPASNoMunicipio,STRING,"Por programas, serviços, projetos e benefícios"
2603,pct_2604,2021-03-04,o1-semestral municipio,comoEstaOrganizadoOPPADaPASNoMunicipio,STRING,Por nível de proteção social
2604,pct_2605,2021-01-09,o1-semestral municipio,comoEstaOrganizadoOPPADaPASNoMunicipio,STRING,Por nível de proteção social
2605,pct_2606,2021-07-01,o1-semestral municipio,comoEstaOrganizadoOPPADaPASNoMunicipio,STRING,"Por programas, serviços, projetos e benefícios"


### E.5 Vigilância Socioassistencial - VS

In [24]:
# métrica Vigilância SocioAssistencial
lst_vls = ('false', 'true')
colunas = ['protocolo', 'dt_protocolo', 'servico']

for i in VS:
    df_campos_neg = df_pct[(df_pct['servico'] == 'o1-semestral municipio')][colunas]
    df_campos_neg['atributo'] = i
    df_campos_neg['tipo'] = 'STRING'
    df_campos_neg['valor'] = df_campos_neg.apply(lambda row: rd.sample(lst_vls, k=1)[0], axis=1)
    df_final = pd.concat([df_final, df_campos_neg])

df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_1,2018-07-01,o1-semestral municipio,col_1,STRING,valor_288
1,pct_1,2018-07-01,o1-semestral municipio,col_2,STRING,valor_260
2,pct_1,2018-07-01,o1-semestral municipio,col_3,STRING,valor_41
3,pct_1,2018-07-01,o1-semestral municipio,col_4,STRING,valor_420
4,pct_1,2018-07-01,o1-semestral municipio,col_5,STRING,valor_330
...,...,...,...,...,...,...
2602,pct_2603,2021-01-01,o1-semestral municipio,oMunicipioDesenvolveQuaisAcoesDeVigilanciaSoci...,STRING,false
2603,pct_2604,2021-03-04,o1-semestral municipio,oMunicipioDesenvolveQuaisAcoesDeVigilanciaSoci...,STRING,true
2604,pct_2605,2021-01-09,o1-semestral municipio,oMunicipioDesenvolveQuaisAcoesDeVigilanciaSoci...,STRING,true
2605,pct_2606,2021-07-01,o1-semestral municipio,oMunicipioDesenvolveQuaisAcoesDeVigilanciaSoci...,STRING,false


### E.6 Monitoramento da Alimentação de Dados da Rede SUAS

In [25]:
# Monitoramento de Sistemas
lst_vls = ('false', 'true')
colunas = ['protocolo', 'dt_protocolo', 'servico']

for i in MONITORA_SISTEMAS:
    df_campos_neg = df_pct[(df_pct['servico'] == 'o1-semestral municipio')][colunas]
    df_campos_neg['atributo'] = i
    df_campos_neg['tipo'] = 'STRING'
    df_campos_neg['valor'] = df_campos_neg.apply(lambda row: rd.sample(lst_vls, k=1)[0], axis=1)
    df_final = pd.concat([df_final, df_campos_neg])

df_final


,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_1,2018-07-01,o1-semestral municipio,col_1,STRING,valor_288
1,pct_1,2018-07-01,o1-semestral municipio,col_2,STRING,valor_260
2,pct_1,2018-07-01,o1-semestral municipio,col_3,STRING,valor_41
3,pct_1,2018-07-01,o1-semestral municipio,col_4,STRING,valor_420
4,pct_1,2018-07-01,o1-semestral municipio,col_5,STRING,valor_330
...,...,...,...,...,...,...
2602,pct_2603,2021-01-01,o1-semestral municipio,quemAlimentaOsDadosNosSistemasDaRedeSUAS-Alime...,STRING,false
2603,pct_2604,2021-03-04,o1-semestral municipio,quemAlimentaOsDadosNosSistemasDaRedeSUAS-Alime...,STRING,false
2604,pct_2605,2021-01-09,o1-semestral municipio,quemAlimentaOsDadosNosSistemasDaRedeSUAS-Alime...,STRING,false
2605,pct_2606,2021-07-01,o1-semestral municipio,quemAlimentaOsDadosNosSistemasDaRedeSUAS-Alime...,STRING,true


### E.7 Alimentação de Dados da Rede SUAS

In [26]:
# Alimentação de Dados no sistema da Rede SUAS
lst_vls = ('false', 'true')
colunas = ['protocolo', 'dt_protocolo', 'servico']

for i in ALIMENTA_SUAS:
    df_campos_neg = df_pct[(df_pct['servico'] == 'o1-semestral municipio')][colunas]
    df_campos_neg['atributo'] = i
    df_campos_neg['tipo'] = 'STRING'
    df_campos_neg['valor'] = df_campos_neg.apply(lambda row: rd.sample(lst_vls, k=1)[0], axis=1)
    df_final = pd.concat([df_final, df_campos_neg])

df_final

,protocolo,dt_protocolo,servico,atributo,tipo,valor
0,pct_1,2018-07-01,o1-semestral municipio,col_1,STRING,valor_288
1,pct_1,2018-07-01,o1-semestral municipio,col_2,STRING,valor_260
2,pct_1,2018-07-01,o1-semestral municipio,col_3,STRING,valor_41
3,pct_1,2018-07-01,o1-semestral municipio,col_4,STRING,valor_420
4,pct_1,2018-07-01,o1-semestral municipio,col_5,STRING,valor_330
...,...,...,...,...,...,...
2602,pct_2603,2021-01-01,o1-semestral municipio,quemAlimentaOsDadosNosSistemasDaRedeSUAS-prote...,STRING,true
2603,pct_2604,2021-03-04,o1-semestral municipio,quemAlimentaOsDadosNosSistemasDaRedeSUAS-prote...,STRING,false
2604,pct_2605,2021-01-09,o1-semestral municipio,quemAlimentaOsDadosNosSistemasDaRedeSUAS-prote...,STRING,true
2605,pct_2606,2021-07-01,o1-semestral municipio,quemAlimentaOsDadosNosSistemasDaRedeSUAS-prote...,STRING,true


### E.8 Equipe que compões setor de VS

In [31]:
EQUIPE_VS

({'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial',
  'suf_nome': 'nome'},
 {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial',
  'suf_escolaridade': 'escolaridade'},
 {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial',
  'suf_formacao': 'formacao'},
 {'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial',
  'suf_cargo': 'cargo'})

In [38]:
# Equipe do Setor de VS
colunas = ['protocolo', 'dt_protocolo', 'servico']

for i in EQUIPE_VS:
    print(i)

{'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 'suf_nome': 'nome'}
{'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 'suf_escolaridade': 'escolaridade'}
{'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 'suf_formacao': 'formacao'}
{'prefixo': 'qualEquipeCompoeOSetorDaVigilanciaSocioassistencial', 'suf_cargo': 'cargo'}


In [39]:
# estipula a quantidade de profissionais que vão compor a equipe do setor de VS
n = rd.randint(N_EQUIPE_VS[0], N_EQUIPE_VS[1])
lst_tudo = []
# monta a estrutura de profissionais e seus atributos
for i in range(n):
    lst_linha = []
    lst_linha.append(names.get_full_name())
    lst_linha.append(rd.sample(lst_escolaridade, k=1)[0][0])
    lst_linha.append(rd.sample(lst_formacao, k=1)[0][0])
    lst_linha.append(rd.sample(lst_cargo, k=1)[0][0])
    lst_linha.append(rd.randint(1, 8))
    
    lst_tudo.append(lst_linha)
    

In [40]:
lst_tudo

[['Henry Goetz', 'Médio - Incompleto', 'Engenharia de Minas', 'Padeiro ', 5],
 ['Anna Raney',
  'Médio - Completo',
  'Engenharia Cartográfica e de Agrimensura',
  'Biomédico Bioquímico ',
  1],
 ['Michael Agosto',
  'Superior - Incompleto',
  'Engenharia Florestal',
  'Auxiliar de Limpeza ',
  2],
 ['David Clark',
  'Médio - Incompleto',
  'Engenharia Cartográfica e de Agrimensura',
  'Auxiliar de Almoxarifado ',
  3],
 ['Kevin Muniz',
  'Pós-graduação (nível mestrado) - Incompleto',
  'Engenharia e Produção',
  'Mecanico ',
  7]]

# <font color='blue'>5.0 carga</font>

In [ ]:
# grava tabelas
df_final.to_csv(caminho_out + 'dataset_sedes.csv', index=False)

# TESTES

In [ ]:
import names

In [ ]:
print(names.get_full_name())

print(names.get_full_name(gender='male'))

print(names.get_first_name())

print(names.get_first_name(gender='female'))

print(names.get_last_name())


In [ ]:
for i in range(20):
    print(names.get_full_name())